# Imports

**NOTE: Make sure to use the get_properties_environment file to set your conda environment.**

In [1]:
import os,re,sys,pickle,datetime,time,random,itertools,glob
import warnings
warnings.filterwarnings("ignore")
import openpyxl
import pandas as pd
from rdkit import Chem
import get_properties_functions as gp

#imports and text patterns for the amine specific LP energy and occupancy function
npa_pattern = re.compile("Summary of Natural Population Analysis:")
nbo_os_pattern = re.compile("beta spin orbitals")
nborbs_pattern = "NATURAL BOND ORBITALS (Summary):"
nborbs2_pattern = re.compile("NATURAL BOND ORBITALS (Summary):")

def get_one_lp_energy(dataframe, a_list): #a function to get the NB orbitals for all atoms (a_list, form ["C1", "C4", "O2"]) in a dataframe that contains file name and atom number
    nborbs_dataframe = pd.DataFrame(columns=[]) #define an empty df to place results in
                
    for index, row in dataframe.iterrows(): #iterate over the dataframe 
        try: 
            atomnum_list = [] 
            for atom in a_list: 
                atomnum = row[str(atom)] #the atom number (i.e., 16) to add to the list is the df entry of this row for the labeled atom (i.e., "C1")
                atomnum_list.append(str(atomnum)) #append that to atomnum_list to make a list of the form [16, 17, 29]
            
            log_file = row['log_name'] #read file name from df
            filecont, error = gp.get_filecont(log_file) #read the contents of the log file
            if error != "":
                print(error)
                row_i = {}
                for a in range(0, len(a_list)):
                    entry = {'NBO_charge_'+str(a_list[a]): "no data"}
                    row_i.update(entry)
                nborbs_dataframe = nborbs_dataframe.append(row_i, ignore_index=True)
                continue
        
            nborbsstart = 0
            #this section finds the line (nborbsstart) where the nbo data is located
            for i in range(len(filecont)-1,0,-1):
                if nborbs_pattern in filecont[i]:#search the file content for the phrase which indicates the start of the NB orbitals section 
                    nborbsstart = i   
            if nborbsstart == 0: 
                error = "****no Natural Bond Orbitals found in: " + str(row['log_name']) + ".log"
                print(error)
                row_i = {}
                for a in range(0, len(a_list)):
                    entry = {'NBO_charge_'+str(a_list[a]): "no data"}
                    row_i.update(entry)
                nborbs_dataframe = nborbs_dataframe.append(row_i, ignore_index=True)
                continue
           
            for atom in a_list: 
                k = 0
                atom_num = row[str(atom)]
                for j in range(nborbsstart,len(filecont)):
                    if str(atom_num) in " ".join(re.findall("([A-Z][a-z]? *[0-9]+)",filecont[j])).split() and ("LP" in filecont[j]):
                        orbital_section = re.search("[0-9]+\.[A-Z\*(0-9 ]+\)",filecont[j]).group(0) #type of MO
                        orbital = orbital_section.split(". ")
                        orb = orbital[1]
                        des = orb.split(" ")
                        orb_type = des[0]
                        occ_energy = [x for x in re.findall(r"[-+]?\d*\.\d+",filecont[j])]
                        occ = occ_energy[0]
                        energy = occ_energy[1]
                        k += 1
                        #print(k)
                if k == 0: 
                    error = "****no LPs for atom " + str(atom)+ " in: " + str(row['log_name']) + ".log"
                    print(error)
                    row_i = {}
                    for atom in a_list:
                        entry = {'NBO_LP_occupancy_' + str(atom): "no data", 'NBO_LP_energy_' + str(atom): "no data"}
                        row_i.update(entry)
                    nborbs_dataframe = nborbs_dataframe.append(row_i, ignore_index=True)
                    pass
                if k == 2: 
                    error = "****more than one LP for atom " + str(atom)+ " in: " + str(row['log_name']) + ".log"
                    print(error)
                    row_i = {}
                    for atom in a_list:
                        entry = {'NBO_LP_occupancy_' + str(atom): "no data", 'NBO_LP_energy_' + str(atom): "no data"}
                        row_i.update(entry)
                    nborbs_dataframe = nborbs_dataframe.append(row_i, ignore_index=True)
                    continue
            
            #this adds the data from the nboout into the new property df
            row_i = {}
            for atom in a_list:
                entry = {'NBO_LP_occupancy_' + str(atom): occ, 'NBO_LP_energy_' + str(atom): energy}
                row_i.update(entry)
            nborbs_dataframe = nborbs_dataframe.append(row_i, ignore_index=True)
        except:
            print('****Unable to acquire NBO orbitals for:', row['log_name'], ".log")
            row_i = {}
            for a in range(0, len(a_list)):
                entry = {'NBO_charge_'+str(a_list[a]): "no data"}
                row_i.update(entry)
            nborbs_dataframe = nborbs_dataframe.append(row_i, ignore_index=True)
    print("NBOrbs function has completed for", a_list)
    return(pd.concat([dataframe, nborbs_dataframe], axis = 1))

D3 import failed


# Atom Inputs Dataframe

## Generate dataframe with atom numbers

### Use command line or bash script to prepare files

To create files: navigate to folder that contains all the log files you wish to analyze.

    obabel *.log -osdf -m  
    ls *.log > log_ids.txt
    cat *.sdf >> molecules.sdf

### Define SMARTS substructure


Recommended to draw the common substructure (with general atoms) in Chemdraw and copy as SMILES (this will generate a SMARTS string)

More information about SMARTS and available characters here: https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html


In [2]:
#this is the common smarts substructure for the molecules you will analyze
#you have to explicitly draw hydrogens into the SMARTS structure if you want to collect properties for hydrogen atoms
substructure = Chem.MolFromSmarts('[H]N([*])[H]')

### Generate preliminary dataframe

In [3]:
#generate a list of molecules using RDkit
all_compounds = Chem.SDMolSupplier('molecules.sdf', removeHs=False) 
#molecules.sdf is generated with the instructions above
#it is a single sdf that contains the structures/atom numbers etc. for every molecule you will analyze

#uses RDKit to search for the substructure in each compound you will analyze
atoms = []
for molecule in all_compounds:
    if molecule is not None:
        submatch = molecule.GetSubstructMatches(substructure) #find substructure
        matchlist = list([item for sublist in submatch for item in sublist]) #list of zero-indexed atom numbers
        match_idx = [x+1 for x in matchlist] #this line changes from 0-indexed to 1-indexed (for Gaussian)
        atoms.append(match_idx) #append 1-indexed list to atoms (a list of lists)
        
#this loop extracts log names from log_ids and splits them to the desired format
filenames = open("log_ids.txt", "r") #generate this with instruction above
#it is a text file that contains the file name for every molecule you will analyze
list_of_filenames = [(line.strip()).split() for line in filenames] #list of the file names (each of which includes all conformers)
list_of_files = []
for filename in list_of_filenames:
    file = filename[0].split(".")
    list_of_files.append(file[0])
filenames.close()

#put the atom numbers for the substructure for each log file into a dataframe
prelim_df = pd.DataFrame(atoms) 
index=list_of_files
prelim_df.insert(0,column='log_name',value=list_of_files)
display(prelim_df)

,log_name,0,1,2,3
0,N1_conf-1,6,1,2,7
1,N1_conf-2,6,1,2,7
2,N1_conf-3,6,1,2,7
3,N1_conf-4,6,1,2,7
4,N1_conf-5,6,1,2,7
5,N1_conf-6,6,1,2,7
6,N1_conf-7,6,1,2,7
7,N1_conf-8,6,1,2,7
8,N2_conf-1,9,1,2,10
9,N2_conf-2,9,1,2,10


### Define column headers using GaussView

Using the preliminary dataframe displayed above, open one of your files and check the atom numbers. 

Assign labels to each column using the cell below. You will call these column headers when you select your properties. 

**User input required:**

In [4]:
atom_labels = {'log_name': 'log_name',
                0: 'H3',
                1: 'N1', 
                2: 'C2',
                3: 'H4'}

### Generate labeled dataframe

**NOTE: it is very important you assign these correctly otherwise the properties you collect will be for the wrong atoms and not produce meaningful correlations.** 

We recommend checking the numbering/headers for at least two different compounds. 

Numbering for different conformers of the same compounds will likely be the same (but may not be for some symmetrical groups).

In [5]:
#rename columns using the user input above
atom_map_df = prelim_df.rename(columns=atom_labels)
display(atom_map_df.head())

#you can use this to clean up the table if you have more atoms in your substructure than you want to collect descriptors for
#atom_map_df = atom_map_df.drop(columns= ['C4', 'C1']) 
#display(atom_map_df.head())

df = atom_map_df #df is what properties will be appended to, this creates a copy so that you have the original preserved 

,log_name,H3,N1,C2,H4
0,N1_conf-1,6,1,2,7
1,N1_conf-2,6,1,2,7
2,N1_conf-3,6,1,2,7
3,N1_conf-4,6,1,2,7
4,N1_conf-5,6,1,2,7


### Save atom map to Excel (if desired)

In [6]:
writer = pd.ExcelWriter('N_example_atom_map.xlsx')
atom_map_df.to_excel(writer)
writer.save()

## Import a manually-generated atom mapping dataframe

If you need to manually generate the atom mapping dataframe, check out the atom_map_sample.xlsx to make sure you have the desired format. 

In [7]:
atom_map_df = pd.read_excel('N_example_atom_map.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(atom_map_df.head())
df = atom_map_df #df is what properties will be appended to, this creates a copy so that you have the original preserved 

,log_name,H3,N1,C2,H4
0,N1_conf-1,6,1,2,7
1,N1_conf-2,6,1,2,7
2,N1_conf-3,6,1,2,7
3,N1_conf-4,6,1,2,7
4,N1_conf-5,6,1,2,7


# Define Properties to Collect

## Collect properties: 

In [8]:
df = atom_map_df

#---------------GoodVibes Engergies---------------
#uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
#calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
#inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

#---------------Frontier Orbitals-----------------
#E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

#---------------Polarizability--------------------
#Exact polarizability
df = gp.get_polarizability(df)

#---------------Dipole----------------------------
#Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

#---------------Volume----------------------------
#Molar volume
#requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

#---------------SASA------------------------------
#Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

#---------------NBO-------------------------------
#natural charge from NBO
#requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["N1", "C2", "H3", "H4"]
df = gp.get_nbo(df, nbo_list) 

#---------------NMR-------------------------------
#isotropic NMR shift
#requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["N1", "C2", "H3", "H4"]
df = gp.get_nmr(df, nmr_list) 

#---------------Pyramidalization------------------
#uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
#collects values based on two definitions of pyramidalization
#details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["N1"]
df = gp.get_pyramidalization(df, pyr_list)

#---------------Lone Pair Energy (custom from first cell)------------------
lp_list = ["N1"]
df = get_one_lp_energy(df, lp_list) 

#---------------Vbur Scan-------------------------
#uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
#inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
#if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["N1"]
df = gp.get_vbur_scan(df, vbur_list, 2, 6, 0.5)

#---------------Sterimol morfeus------------------
#uses morfeus to calculate Sterimol L, B1, and B5 values
#NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["N1", "C2"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists) 



   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using v

In [9]:
#note: because H3 and H4 are equivalent, average the properties of H3 and H4
df['NBO_charge_H_avg'] = df[['NBO_charge_H3', 'NBO_charge_H4']].mean(axis=1)
df['NMR_shift_H_avg'] =  df[['NMR_shift_H3', 'NMR_shift_H4']].mean(axis=1)

#drop the data collected for individual H atoms 
df.drop(['NBO_charge_H3', 'NBO_charge_H4', 'NMR_shift_H3', 'NMR_shift_H4'], axis=1, inplace =True)

#move averaged properties to the NMR and NBO section (rather than the end of the df)
NBO_charge_H_avg = df.pop("NBO_charge_H_avg")
NMR_shift_H_avg = df.pop('NMR_shift_H_avg')
df.insert(26, "NBO_charge_H_avg", NBO_charge_H_avg)
df.insert(29, "NMR_shift_H_avg", NMR_shift_H_avg)

pd.options.display.max_columns = None
display(df)

,log_name,H3,N1,C2,H4,E_spc (Hartree),G(T)_spc(Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,polar_aniso(au),polar_iso(au),dipole(Debye),volume(Bohr_radius³/mol),SASA_sphericity,SASA_surface_area(Å²),SASA_volume(Å³),NBO_charge_C2,NBO_charge_H_avg,NBO_charge_N1,NMR_shift_C2,NMR_shift_H_avg,NMR_shift_N1,pyramidalization_Agranat-Radhakrishnan_N1,pyramidalization_Gavrish_N1(°),NBO_LP_energy_N1,NBO_LP_occupancy_N1,%Vbur_N1_2.0Å,%Vbur_N1_2.5Å,%Vbur_N1_3.0Å,%Vbur_N1_3.5Å,%Vbur_N1_4.0Å,%Vbur_N1_4.5Å,%Vbur_N1_5.0Å,%Vbur_N1_5.5Å,%Vbur_N1_6.0Å,Sterimol_B1_N1_C2(Å)_morfeus,Sterimol_B5_N1_C2(Å)_morfeus,Sterimol_L_N1_C2(Å)_morfeus
0,N1_conf-1,6,1,2,7,-249.671914,-249.575055,-249.537801,298.15,0.037254,0.037052,0.126641,-249.574853,-0.30745,0.06420,0.37165,-0.121625,0.01990,10.07110,50.8234,2.4550,759.459,0.926089,243.329548,318.086208,-0.23268,0.351985,-0.82093,138.7185,31.49780,234.2248,0.818903,6.106775,-0.35794,1.95717,79.971591,61.440292,47.199046,37.000908,28.726879,22.714864,18.151627,14.696973,11.881264,1.700000,5.492973,4.742869
1,N1_conf-2,6,1,2,7,-249.669893,-249.572974,-249.535680,298.15,0.037294,0.036999,0.126761,-249.572680,-0.30423,0.06331,0.36754,-0.120460,0.01974,9.27981,50.1439,1.4564,1010.352,0.938943,237.699378,313.526862,-0.24818,0.351790,-0.82312,143.9264,31.52935,237.1377,0.825241,6.173436,-0.36164,1.95803,79.855372,61.585167,48.038960,38.596604,30.652582,24.519690,19.333591,15.293491,11.945326,1.700000,5.115875,4.830794
2,N1_conf-3,6,1,2,7,-249.669628,-249.573279,-249.535594,298.15,0.037685,0.037494,0.126354,-249.573089,-0.31327,0.06130,0.37457,-0.125985,0.02119,15.20510,50.7761,0.9797,815.735,0.919268,246.769316,321.272663,-0.22815,0.352935,-0.83205,139.6021,31.68915,237.8992,0.794886,5.868374,-0.36686,1.96331,81.156379,61.653536,46.645933,35.980595,27.144498,20.891586,16.692157,13.638860,11.287927,1.914883,4.503247,6.543307
3,N1_conf-4,6,1,2,7,-249.670020,-249.573121,-249.535837,298.15,0.037284,0.036933,0.126752,-249.572770,-0.30600,0.06238,0.36838,-0.121810,0.02014,8.86097,50.0789,2.3011,743.188,0.936516,238.279418,313.455966,-0.24471,0.357620,-0.83604,145.3575,31.37050,237.8530,0.794689,5.864048,-0.35883,1.96006,80.833549,62.364891,48.727093,39.219740,30.954007,24.560394,19.313290,15.277401,11.931267,1.899521,5.113481,4.820416
4,N1_conf-5,6,1,2,7,-249.667820,-249.571505,-249.533683,298.15,0.037822,0.037492,0.126482,-249.571176,-0.31350,0.06014,0.37364,-0.126680,0.02147,13.75230,50.1302,1.3365,903.838,0.929121,242.119209,317.268080,-0.24367,0.353295,-0.83180,142.1548,31.69160,238.2211,0.792354,5.843836,-0.36825,1.96199,81.243543,61.757716,46.717633,36.258386,28.030926,22.066310,17.818147,14.466692,11.676493,1.921117,3.821203,6.606193
5,N1_conf-6,6,1,2,7,-249.672198,-249.575220,-249.538043,298.15,0.037177,0.036981,0.126705,-249.575024,-0.30636,0.06929,0.37565,-0.118535,0.01870,11.90680,50.8111,0.9951,860.175,0.924636,243.549575,317.768558,-0.22988,0.357785,-0.83482,139.5798,31.28465,234.8215,0.803973,5.955103,-0.35892,1.96155,80.727014,62.237922,48.005438,37.941437,29.429039,23.182691,18.455347,14.870570,11.923330,1.923202,5.495151,4.565530
6,N1_conf-7,6,1,2,7,-249.667619,-249.570742,-249.533576,298.15,0.037166,0.037015,0.126575,-249.570592,-0.31062,0.05941,0.37003,-0.125605,0.02132,7.30648,50.2013,2.0344,887.807,0.938178,237.669340,313.084201,-0.24830,0.354170,-0.83342,142.9556,31.76465,237.0221,0.781832,5.740068,-0.36143,1.95885,82.770532,65.102878,51.387440,41.538739,32.938675,26.144332,20.283182,15.552705,11.971178,1.914817,5.053783,4.608475
7,N1_conf-8,6,1,2,7,-249.666992,-249.571062,-249.533276,298.15,0.037786,0.037516,0.126052,-249.570792,-0.30526,0.06252,0.36778,-0.121370,0.02003,11.15280,51.0284,2.0972,694.084,0.921725,245.609443,320.290282,-0.23088,0.349475,-0.82134,138.2673,31.78970,231.1173,0.791689,5.836066,-0.35820,1.96096,81.227402,62.290012,47.725156,37.490682,28.986407,22.807941,18.171139,14.714835,11.912445,1.931330,5.494813,4.796299
8,N2_conf-1,9,1,2,10,-33

## Save collected properties to Excel

Helpful to save here in case the Notebook crashes or if you want to add more properties before post-processsing. Can be read in at 5.1.1.

In [10]:
writer = pd.ExcelWriter('All_Conformer_Properties_for_N_example.xlsx')
df.to_excel(writer)
writer.save()

# Post-processing

## User input for data processing

In [11]:
#for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
#this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
prefix = "N" 
suffix = "_"

#columns that provide atom mapping information are dropped
atom_columns_to_drop = ["C2", "N1", "H3", "H4"]

#title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

Check out the dataframe_sample.xlsx to make sure you have the desired format. 

In [12]:
df = pd.read_excel('All_Conformer_Properties_for_N_example.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(df.head())

,log_name,H3,N1,C2,H4,E_spc (Hartree),G(T)_spc(Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,polar_aniso(au),polar_iso(au),dipole(Debye),volume(Bohr_radius³/mol),SASA_sphericity,SASA_surface_area(Å²),SASA_volume(Å³),NBO_charge_C2,NBO_charge_H_avg,NBO_charge_N1,NMR_shift_C2,NMR_shift_H_avg,NMR_shift_N1,pyramidalization_Agranat-Radhakrishnan_N1,pyramidalization_Gavrish_N1(°),NBO_LP_energy_N1,NBO_LP_occupancy_N1,%Vbur_N1_2.0Å,%Vbur_N1_2.5Å,%Vbur_N1_3.0Å,%Vbur_N1_3.5Å,%Vbur_N1_4.0Å,%Vbur_N1_4.5Å,%Vbur_N1_5.0Å,%Vbur_N1_5.5Å,%Vbur_N1_6.0Å,Sterimol_B1_N1_C2(Å)_morfeus,Sterimol_B5_N1_C2(Å)_morfeus,Sterimol_L_N1_C2(Å)_morfeus
0,N1_conf-1,6,1,2,7,-249.671914,-249.575055,-249.537801,298.15,0.037254,0.037052,0.126641,-249.574853,-0.30745,0.06420,0.37165,-0.121625,0.01990,10.07110,50.8234,2.4550,759.459,0.926089,243.329548,318.086208,-0.23268,0.351985,-0.82093,138.7185,31.49780,234.2248,0.818903,6.106775,-0.35794,1.95717,79.971591,61.440292,47.199046,37.000908,28.726879,22.714864,18.151627,14.696973,11.881264,1.700000,5.492973,4.742869
1,N1_conf-2,6,1,2,7,-249.669893,-249.572974,-249.535680,298.15,0.037294,0.036999,0.126761,-249.572680,-0.30423,0.06331,0.36754,-0.120460,0.01974,9.27981,50.1439,1.4564,1010.352,0.938943,237.699378,313.526862,-0.24818,0.351790,-0.82312,143.9264,31.52935,237.1377,0.825241,6.173436,-0.36164,1.95803,79.855372,61.585167,48.038960,38.596604,30.652582,24.519690,19.333591,15.293491,11.945326,1.700000,5.115875,4.830794
2,N1_conf-3,6,1,2,7,-249.669628,-249.573279,-249.535594,298.15,0.037685,0.037494,0.126354,-249.573089,-0.31327,0.06130,0.37457,-0.125985,0.02119,15.20510,50.7761,0.9797,815.735,0.919268,246.769316,321.272663,-0.22815,0.352935,-0.83205,139.6021,31.68915,237.8992,0.794886,5.868374,-0.36686,1.96331,81.156379,61.653536,46.645933,35.980595,27.144498,20.891586,16.692157,13.638860,11.287927,1.914883,4.503247,6.543307
3,N1_conf-4,6,1,2,7,-249.670020,-249.573121,-249.535837,298.15,0.037284,0.036933,0.126752,-249.572770,-0.30600,0.06238,0.36838,-0.121810,0.02014,8.86097,50.0789,2.3011,743.188,0.936516,238.279418,313.455966,-0.24471,0.357620,-0.83604,145.3575,31.37050,237.8530,0.794689,5.864048,-0.35883,1.96006,80.833549,62.364891,48.727093,39.219740,30.954007,24.560394,19.313290,15.277401,11.931267,1.899521,5.113481,4.820416
4,N1_conf-5,6,1,2,7,-249.667820,-249.571505,-249.533683,298.15,0.037822,0.037492,0.126482,-249.571176,-0.31350,0.06014,0.37364,-0.126680,0.02147,13.75230,50.1302,1.3365,903.838,0.929121,242.119209,317.268080,-0.24367,0.353295,-0.83180,142.1548,31.69160,238.2211,0.792354,5.843836,-0.36825,1.96199,81.243543,61.757716,46.717633,36.258386,28.030926,22.066310,17.818147,14.466692,11.676493,1.921117,3.821203,6.606193


## Generating a list of compounds that have conformational ensembles

In [13]:
#this is a template for our files that are all named "AcXXX_clust-X.log"

compound_list = []
    
for index, row in df.iterrows():
    log_file = row['log_name'] #read file name from df
    prefix_and_compound = log_file.split(str(suffix)) #splits to get "AcXXX" (entry O) (and we don't use the "clust-X" (entry 1))
    compound = prefix_and_compound[0].split(str(prefix)) #splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
    compound_list.append(compound[1])

compound_list = list(set(compound_list)) #removes duplicate stuctures that result from having conformers of each
compound_list.sort() #reorders numerically (not sure if it reorders alphabetically)
print(compound_list)

#this should generate a list that looks like this: ['24', '27', '34', '48']

['1', '2', '3', '4', '5']


## Post-processing to get properties for each compound

In [14]:
all_df_master = pd.DataFrame(columns=[])
properties_df_master = pd.DataFrame(columns=[])

for compound in compound_list: 
    #defines the common start to all files using the input above 
    substring = str(prefix) + str(compound) + str(suffix)
    
    #makes a data frame for one compound at a time for post-processing
    valuesdf = df[df["log_name"].str.startswith(substring)]
    valuesdf = valuesdf.drop(columns = atom_columns_to_drop)
    valuesdf = valuesdf.reset_index(drop = True)  #you must re-index otherwise the 2nd, 3rd, etc. compounds fail
   
    #define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
    non_boltz_columns = ["G(Hartree)","∆G(Hartree)","∆G(kcal/mol)", "e^(-∆G/RT)","Mole Fraction"] #don't boltzman average columns containing these strings in the column label
    reg_avg_columns = ['CPU_time_total(hours)', 'Wall_time_total(hours)'] #don't boltzmann average these either, we average them in case that is helpful
    gv_extra_columns = ['E_spc (Hartree)', 'H_spc(Hartree)', 'T', 'T*S', 'T*qh_S', 'ZPE(Hartree)', 'qh_G(T)_spc(Hartree)', "G(T)_spc(Hartree)"]
    gv_extra_columns.remove(str(energy_col_header))
    
    #calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
    valuesdf["∆G(Hartree)"] = valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
    valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
    valuesdf["e^(-∆G/RT)"] = np.exp((valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)) #R is in cal/(K*mol)
    valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
    values_boltz_row = []
    values_min_row = []
    values_max_row = []
    values_boltz_stdev_row =[]
    #values_range_row = []
    values_exclude_columns = []
    
    for column in valuesdf:
        if "log_name" in column:
            values_boltz_row.append("Boltzmann Averages")
            values_min_row.append("Ensemble Minimum")
            values_max_row.append("Ensemble Maximum")
            values_boltz_stdev_row.append("Boltzmann Standard Deviation")
            #values_range_row.append("Ensemble Range")
            values_exclude_columns.append(column) #used later to build final dataframe
        elif any(phrase in column for phrase in non_boltz_columns) or any(phrase in column for phrase in gv_extra_columns):
            values_boltz_row.append("")
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            #values_range_row.append("")
        elif any(phrase in column for phrase in reg_avg_columns):
            values_boltz_row.append(valuesdf[column].mean()) #intended to print the average CPU/wall time in the boltz column
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            #values_range_row.append("")
        else:
            valuesdf[column] = pd.to_numeric(valuesdf[column]) #to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
            values_boltz_row.append((valuesdf[column] * valuesdf["Mole Fraction"]).sum())
            values_min_row.append(valuesdf[column].min())
            values_max_row.append(valuesdf[column].max())
            #values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

            
            # this section generates the weighted std deviation (weighted by mole fraction) 
            # formula: https://www.statology.org/weighted-standard-deviation-excel/
    
            boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum() #number
            delta_values_sq = []
    
            #makes a list of the "deviation" for each conformer           
            for index, row in valuesdf.iterrows(): 
                value = row[column]
                delta_value_sq = (value - boltz)**2
                delta_values_sq.append(delta_value_sq)
            
            #w is list of weights (i.e. mole fractions)
            w = list(valuesdf["Mole Fraction"])
            wstdev = np.sqrt( (np.average(delta_values_sq, weights=w)) / (((len(w)-1)/len(w))*np.sum(w)) )
            #np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
           
            #if there is only one conformer in the ensemble, set the weighted standard deviation to 0 
            if len(w) == 1:
                wstdev = 0
            values_boltz_stdev_row.append(wstdev)
            
            
    valuesdf.loc[len(valuesdf)] = values_boltz_row
    valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
    valuesdf.loc[len(valuesdf)] = values_min_row
    valuesdf.loc[len(valuesdf)] = values_max_row
    #valuesdf.loc[len(valuesdf)] = values_range_row

    #final output format is built here:
    explicit_order_front_columns = ["log_name", energy_col_header,"∆G(Hartree)","∆G(kcal/mol)","e^(-∆G/RT)","Mole Fraction"]
    
    #reorders the dataframe using front columns defined above
    valuesdf = valuesdf[explicit_order_front_columns + [col for col in valuesdf.columns if col not in explicit_order_front_columns and col not in values_exclude_columns]]
    
    #determine the index of the lowest energy conformer
    low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Lowest E Conformer
    new_row = valuesdf.loc[low_e_index[0]]
    new_row['log_name'] = "Lowest E Conformer"   
    valuesdf =  valuesdf.append(new_row, ignore_index=True)

#------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------  
    #if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
    #this template can be adjusted for min/max/etc. 
    
    #find the index for the min or max column:
    #ensemble_min_index = valuesdf[valuesdf["log_name"] == "Ensemble Minimum"].index.tolist()
    
    #find the min or max value of the property (based on index above)
    #saves the value in a list (min_value) with one entry (this is why we call min_value[0])
    #min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C4_3.0Å"].tolist()   
    #vbur_min_index = valuesdf[valuesdf["%Vbur_C4_3.0Å"] == min_value[0]].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Property_min_conformer
    #new_row = valuesdf.loc[vbur_min_index[0]]
    #new_row['log_name'] = "%Vbur_C4_3.0Å_min_Conformer"   
    #valuesdf =  valuesdf.append(new_row, ignore_index=True)
#--------------------------------------------------------------------------------------------------------------------    
    
    #appends the frame to the master output
    all_df_master = pd.concat([all_df_master, valuesdf])
    
    #drop all the individual conformers
    dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
    valuesdf = valuesdf.drop(dropindex)
    valuesdf = valuesdf.reset_index(drop = True)
    
    #display(valuesdf)   
    
    #drop the columns created to determine the mole fraction and some that 
    valuesdf = valuesdf.drop(columns = explicit_order_front_columns)
    try:
        valuesdf = valuesdf.drop(columns = gv_extra_columns)
    except:
        pass
    try:
        valuesdf = valuesdf.drop(columns = reg_avg_columns)
    except:
        pass
        
#---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------  
    compound_name = prefix + str(compound) 
#--------------------------------------------------------------------------------------------------------------------      

    properties_df = pd.DataFrame({'Compound_Name': [compound_name]})
    
    #builds a dataframe (for each compound) by adding summary properties as new columns
    for (columnName, columnData) in valuesdf.iteritems():
        #the indexes need to match the values dataframe - display it to double check if you need to make changes 
        #(uncomment the display(valuesdf) in row 124 of this cell)
        properties_df[str(columnName) + "_Boltz"] = [columnData.values[0]]
        properties_df[str(columnName) + "_Boltz_stdev"] = [columnData.values[1]]
        properties_df[str(columnName) + "_min"] = [columnData.values[2]]
        properties_df[str(columnName) + "_max"] = [columnData.values[3]]
        #properties_df[str(columnName) + "_range"] = [columnData.values[4]]
        properties_df[str(columnName) + "_low_E"] = [columnData.values[4]] 
        
        #if you're collecting properties for a specific conformer, add these here (note the index)
        #example:
        #properties_df[str(columnName) + "_V_bur_min"] = [columnData.values[6]]
        
        #if you only want a table with Boltz, you can comment out the other summary properties to generate a Boltz spreadsheet
        #of if you don't want to collect range, etc.
    #concatenates the individual acid properties df into the master properties df
    properties_df_master = pd.concat([properties_df_master, properties_df], axis = 0)

all_df_master = all_df_master.reset_index(drop = True)
properties_df_master = properties_df_master.reset_index(drop = True)


### Peek at your new dataframes

In [15]:
display(properties_df_master.head())
display(all_df_master)

,Compound_Name,HOMO_Boltz,HOMO_Boltz_stdev,HOMO_min,HOMO_max,HOMO_low_E,LUMO_Boltz,LUMO_Boltz_stdev,LUMO_min,LUMO_max,LUMO_low_E,η_Boltz,η_Boltz_stdev,η_min,η_max,η_low_E,μ_Boltz,μ_Boltz_stdev,μ_min,μ_max,μ_low_E,ω_Boltz,ω_Boltz_stdev,ω_min,ω_max,ω_low_E,polar_aniso(au)_Boltz,polar_aniso(au)_Boltz_stdev,polar_aniso(au)_min,polar_aniso(au)_max,polar_aniso(au)_low_E,polar_iso(au)_Boltz,polar_iso(au)_Boltz_stdev,polar_iso(au)_min,polar_iso(au)_max,polar_iso(au)_low_E,dipole(Debye)_Boltz,dipole(Debye)_Boltz_stdev,dipole(Debye)_min,dipole(Debye)_max,dipole(Debye)_low_E,volume(Bohr_radius³/mol)_Boltz,volume(Bohr_radius³/mol)_Boltz_stdev,volume(Bohr_radius³/mol)_min,volume(Bohr_radius³/mol)_max,volume(Bohr_radius³/mol)_low_E,SASA_sphericity_Boltz,SASA_sphericity_Boltz_stdev,SASA_sphericity_min,SASA_sphericity_max,SASA_sphericity_low_E,SASA_surface_area(Å²)_Boltz,SASA_surface_area(Å²)_Boltz_stdev,SASA_surface_area(Å²)_min,SASA_surface_area(Å²)_max,SASA_surface_area(Å²)_low_E,SASA_volume(Å³)_Boltz,SASA_volume(Å³)_Boltz_stdev,SASA_volume(Å³)_min,SASA_volume(Å³)_max,SASA_volume(Å³)_low_E,NBO_charge_C2_Boltz,NBO_charge_C2_Boltz_stdev,NBO_charge_C2_min,NBO_charge_C2_max,NBO_charge_C2_low_E,NBO_charge_H_avg_Boltz,NBO_charge_H_avg_Boltz_stdev,NBO_charge_H_avg_min,NBO_charge_H_avg_max,NBO_charge_H_avg_low_E,NBO_charge_N1_Boltz,NBO_charge_N1_Boltz_stdev,NBO_charge_N1_min,NBO_charge_N1_max,NBO_charge_N1_low_E,NMR_shift_C2_Boltz,NMR_shift_C2_Boltz_stdev,NMR_shift_C2_min,NMR_shift_C2_max,NMR_shift_C2_low_E,NMR_shift_H_avg_Boltz,NMR_shift_H_avg_Boltz_stdev,NMR_shift_H_avg_min,NMR_shift_H_avg_max,NMR_shift_H_avg_low_E,NMR_shift_N1_Boltz,NMR_shift_N1_Boltz_stdev,NMR_shift_N1_min,NMR_shift_N1_max,NMR_shift_N1_low_E,pyramidalization_Agranat-Radhakrishnan_N1_Boltz,pyramidalization_Agranat-Radhakrishnan_N1_Boltz_stdev,pyramidalization_Agranat-Radhakrishnan_N1_min,pyramidalization_Agranat-Radhakrishnan_N1_max,pyramidalization_Agranat-Radhakrishnan_N1_low_E,pyramidalization_Gavrish_N1(°)_Boltz,pyramidalization_Gavrish_N1(°)_Boltz_stdev,pyramidalization_Gavrish_N1(°)_min,pyramidalization_Gavrish_N1(°)_max,pyramidalization_Gavrish_N1(°)_low_E,NBO_LP_energy_N1_Boltz,NBO_LP_energy_N1_Boltz_stdev,NBO_LP_energy_N1_min,NBO_LP_energy_N1_max,NBO_LP_energy_N1_low_E,NBO_LP_occupancy_N1_Boltz,NBO_LP_occupancy_N1_Boltz_stdev,NBO_LP_occupancy_N1_min,NBO_LP_occupancy_N1_max,NBO_LP_occupancy_N1_low_E,%Vbur_N1_2.0Å_Boltz,%Vbur_N1_2.0Å_Boltz_stdev,%Vbur_N1_2.0Å_min,%Vbur_N1_2.0Å_max,%Vbur_N1_2.0Å_low_E,%Vbur_N1_2.5Å_Boltz,%Vbur_N1_2.5Å_Boltz_stdev,%Vbur_N1_2.5Å_min,%Vbur_N1_2.5Å_max,%Vbur_N1_2.5Å_low_E,%Vbur_N1_3.0Å_Boltz,%Vbur_N1_3.0Å_Boltz_stdev,%Vbur_N1_3.0Å_min,%Vbur_N1_3.0Å_max,%Vbur_N1_3.0Å_low_E,%Vbur_N1_3.5Å_Boltz,%Vbur_N1_3.5Å_Boltz_stdev,%Vbur_N1_3.5Å_min,%Vbur_N1_3.5Å_max,%Vbur_N1_3.5Å_low_E,%Vbur_N1_4.0Å_Boltz,%Vbur_N1_4.0Å_Boltz_stdev,%Vbur_N1_4.0Å_min,%Vbur_N1_4.0Å_max,%Vbur_N1_4.0Å_low_E,%Vbur_N1_4.5Å_Boltz,%Vbur_N1_4.5Å_Boltz_stdev,%Vbur_N1_4.5Å_min,%Vbur_N1_4.5Å_max,%Vbur_N1_4.5Å_low_E,%Vbur_N1_5.0Å_Boltz,%Vbur_N1_5.0Å_Boltz_stdev,%Vbur_N1_5.0Å_min,%Vbur_N1_5.0Å_max,%Vbur_N1_5.0Å_low_E,%Vbur_N1_5.5Å_Boltz,%Vbur_N1_5.5Å_Boltz_stdev,%Vbur_N1_5.5Å_min,%Vbur_N1_5.5Å_max,%Vbur_N1_5.5Å_low_E,%Vbur_N1_6.0Å_Boltz,%Vbur_N1_6.0Å_Boltz_stdev,%Vbur_N1_6.0Å_min,%Vbur_N1_6.0Å_max,%Vbur_N1_6.0Å_low_E,Sterimol_B1_N1_C2(Å)_morfeus_Boltz,Sterimol_B1_N1_C2(Å)_morfeus_Boltz_stdev,Sterimol_B1_N1_C2(Å)_morfeus_min,Sterimol_B1_N1_C2(Å)_morfeus_max,Sterimol_B1_N1_C2(Å)_morfeus_low_E,Sterimol_B5_N1_C2(Å)_morfeus_Boltz,Sterimol_B5_N1_C2(Å)_morfeus_Boltz_stdev,Sterimol_B5_N1_C2(Å)_morfeus_min,Sterimol_B5_N1_C2(Å)_morfeus_max,Sterimol_B5_N1_C2(Å)_morfeus_low_E,Sterimol_L_N1_C2(Å)_morfeus_Boltz,Sterimol_L_N1_C2(Å)_morfeus_Boltz_stdev,Sterimol_L_N1_C2(Å)_morfeus_min,Sterimol_L_N1_C2(Å)_morfeus_max,Sterimol_L_N1_C2(Å)_morfeus_low_E
0,N1,-0.307142,0.001952,-0.31350,-0.30423,-0.30636,0.066145,0.003185,0.05941,0.06929,0.06929,0.373287,0.002708,0.36754,0.37565,0.37565,-0.120499,0.002268,-0.

,log_name,G(T)_spc(Hartree),∆G(Hartree),∆G(kcal/mol),e^(-∆G/RT),Mole Fraction,E_spc (Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,polar_aniso(au),polar_iso(au),dipole(Debye),volume(Bohr_radius³/mol),SASA_sphericity,SASA_surface_area(Å²),SASA_volume(Å³),NBO_charge_C2,NBO_charge_H_avg,NBO_charge_N1,NMR_shift_C2,NMR_shift_H_avg,NMR_shift_N1,pyramidalization_Agranat-Radhakrishnan_N1,pyramidalization_Gavrish_N1(°),NBO_LP_energy_N1,NBO_LP_occupancy_N1,%Vbur_N1_2.0Å,%Vbur_N1_2.5Å,%Vbur_N1_3.0Å,%Vbur_N1_3.5Å,%Vbur_N1_4.0Å,%Vbur_N1_4.5Å,%Vbur_N1_5.0Å,%Vbur_N1_5.5Å,%Vbur_N1_6.0Å,Sterimol_B1_N1_C2(Å)_morfeus,Sterimol_B5_N1_C2(Å)_morfeus,Sterimol_L_N1_C2(Å)_morfeus
0,N1_conf-1,-249.575055,0.000165,0.103281,0.84003,0.380169,-249.671914,-249.537801,298.15,0.037254,0.037052,0.126641,-249.574853,-0.307450,0.064200,0.371650,-0.121625,0.019900,10.071100,50.823400,2.455000,759.459000,0.926089,243.329548,318.086208,-0.232680,0.351985,-0.820930,138.718500,31.497800,234.224800,0.818903,6.106775,-0.357940,1.957170,79.971591,61.440292,47.199046,37.000908,28.726879,22.714864,18.151627,14.696973,11.881264,1.700000,5.492973,4.742869
1,N1_conf-2,-249.572974,0.002246,1.40912,0.092706,0.041956,-249.669893,-249.53568,298.15,0.037294,0.036999,0.126761,-249.57268,-0.304230,0.063310,0.367540,-0.120460,0.019740,9.279810,50.143900,1.456400,1010.352000,0.938943,237.699378,313.526862,-0.248180,0.351790,-0.823120,143.926400,31.529350,237.137700,0.825241,6.173436,-0.361640,1.958030,79.855372,61.585167,48.038960,38.596604,30.652582,24.519690,19.333591,15.293491,11.945326,1.700000,5.115875,4.830794
2,N1_conf-3,-249.573279,0.001941,1.217672,0.128068,0.057959,-249.669628,-249.535594,298.15,0.037685,0.037494,0.126354,-249.573089,-0.313270,0.061300,0.374570,-0.125985,0.021190,15.205100,50.776100,0.979700,815.735000,0.919268,246.769316,321.272663,-0.228150,0.352935,-0.832050,139.602100,31.689150,237.899200,0.794886,5.868374,-0.366860,1.963310,81.156379,61.653536,46.645933,35.980595,27.144498,20.891586,16.692157,13.638860,11.287927,1.914883,4.503247,6.543307
3,N1_conf-4,-249.573121,0.002099,1.316925,0.108315,0.049019,-249.67002,-249.535837,298.15,0.037284,0.036933,0.126752,-249.57277,-0.306000,0.062380,0.368380,-0.121810,0.020140,8.860970,50.078900,2.301100,743.188000,0.936516,238.279418,313.455966,-0.244710,0.357620,-0.836040,145.357500,31.370500,237.853000,0.794689,5.864048,-0.358830,1.960060,80.833549,62.364891,48.727093,39.219740,30.954007,24.560394,19.313290,15.277401,11.931267,1.899521,5.113481,4.820416
4,N1_conf-5,-249.571505,0.003715,2.331002,0.01956,0.008852,-249.66782,-249.533683,298.15,0.037822,0.037492,0.126482,-249.571176,-0.313500,0.060140,0.373640,-0.126680,0.021470,13.752300,50.130200,1.336500,903.838000,0.929121,242.119209,317.268080,-0.243670,0.353295,-0.831800,142.154800,31.691600,238.221100,0.792354,5.843836,-0.368250,1.961990,81.243543,61.757716,46.717633,36.258386,28.030926,22.066310,17.818147,14.466692,11.676493,1.921117,3.821203,6.606193
5,N1_conf-6,-249.57522,0.0,0.0,1.0,0.452566,-249.672198,-249.538043,298.15,0.037177,0.036981,0.126705,-249.575024,-0.306360,0.069290,0.375650,-0.118535,0.018700,11.906800,50.811100,0.995100,860.175000,0.924636,243.549575,317.768558,-0.229880,0.357785,-0.834820,139.579800,31.284650,234.821500,0.803973,5.955103,-0.358920,1.961550,80.727014,62.237922,48.005438,37.941437,29.429039,23.182691,18.455347,14.870570,11.923330,1.923202,5.495151,4.565530
6,N1_conf-7,-249.570742,0.004478,2.80979,0.008718,0.003945,-249.667619,-249.533576,298.15,0.037166,0.037015,0.126575,-249.570592,-0.310620,0.059410,0.370030,-0.125605,0.021320,7.306480,50.201300,2.034400,887.807000,0.938178,237.669340,313.084201,-0.248300,0.354170,-0.833420,142.955600,31.764650,237.022100,0.781832,5.740068,-0.361430,1.958850,82.770532,65.102878,51.387440,41.538739,32.938675,26.144332,20.283182,15.552705,11.971178,1.914817,5.053783,4.608475
7,N1_conf-8,-249.571062,0.004158,2.609374,0.012227,0.005533,-249.666992,-249.53327

### Save to Microsoft Excelᵀᴹ 

In [16]:
all_df_master.to_excel('All_Conformer_and_Summary_Properties_for_N_example.xlsx', index = False)
properties_df_master.to_excel('Summary_Properties_for_N_example.xlsx', index = False)